In [1]:
# Author: Andrew Laws
# Date: 3/22/2021

#Set environment
arcpy.env.workspace = "C:\\Users\\alaws2\\OneDrive - University of Nebraska-Lincoln\\Spring 2021 Courses\\PSA for GIS\\Lab4"

#Set original datadata
lancaster =  "lancaster_county.shp"
stpark = 'State_Park_Locations.shp'
munic = 'Municipal_Boundaries.shp'
streams = "Streams_303_d_.shp"
dem = "ned30lc.tif"
landco = "nlcd_lc_14n.tif"

In [ ]:
# Suitability Critiera

# Within 2 km of a state park
# 1 km or more away from a 303d stream
# More than 5 km from municipal boundaries
# More than 401 meters in elevation
# On areas of “moderate” slope

In [8]:
#Copy DEM and bring in copy
dem_suit = ".\\dem_suit.tif"
dem = arcpy.Raster(dem)
dem.readOnly = False

#Remap copy so suitable habitat = 1
myRemapRange = arcpy.sa.RemapRange([[400, 470, 1], [0, 400, 0]])
demsuit = arcpy.sa.Reclassify(dem, "VALUE", myRemapRange)
demsuit.save(dem_suit)

In [9]:
#Create slope from DEM
slope = ".\slope.tif"
dem = arcpy.Raster(dem)
dem.readOnly = False
outslope = arcpy.sa.Slope(dem)
outslope.save(slope)

#Save copy of slope for reclassifying suitability
slope_int = ".\slope_int.tif"
outslope.save(slope_int)

#Bring in slope suitablity raster
slope_int = arcpy.Raster(slope_int)
slope_int.readOnly = False

#Reclassify slope as moderate slopes (> 5 degrees) = 1
slope_suit = ".\slope_suit.tif"
myRemapRange = arcpy.sa.RemapRange([[0, 5, 0], [5, 20, 1]])
slopesuit = arcpy.sa.Reclassify(slope_int, "VALUE", myRemapRange)
slopesuit.save(slope_suit)

In [19]:
#Select state parks in Lancaster County
stpark_sel = arcpy.management.SelectLayerByLocation(stpark, "INTERSECT", lancaster)

#Buffer parks by 2 km to memory layer 
stparkbuff = "stpark_buff.shp"
stpark_buff = arcpy.analysis.Buffer(stpark_sel, stparkbuff, "2 Kilometers", dissolve_option = 'ALL')

#Erase streambuff from Lancaster
stparkclip = "stpark_clip.shp"
arcpy.analysis.Erase(lancaster, stpark_buff, stparkclip)

#Convert clip to raster
stpark_int = ".\stpark_int.tif"
arcpy.conversion.PolygonToRaster(stparkclip, "FID", stpark_int)

#Reclassify stpark suit
stparkint = arcpy.Raster(stpark_int)
stparkint.readOnly = False

stparksuit = arcpy.sa.Reclassify(stparkint, "Value", "0 0;NODATA 1")
stpark_suit = ".\stpark_suit.tif"
stparksuit.save(stpark_suit)

arcpy.management.Delete(stpark_sel)

<Result 'true'>

In [14]:
#Select streams in Lancaster County
stream_sel = arcpy.management.SelectLayerByLocation(streams, "INTERSECT", lancaster)

#Buffer streams by 1 km
streambuff = "stream_buff.shp"
stream_buff = arcpy.analysis.Buffer(stream_sel, streambuff, "1 Kilometers", dissolve_option = 'ALL')

#Erase streambuff from Lancaster
streamclip = "stream_clip.shp"
arcpy.analysis.Erase(lancaster, stream_buff, streamclip)

#Convert clip to raster
stream_int = ".\stream_int.tif"
arcpy.conversion.PolygonToRaster(streamclip, "FID", stream_int)

#Reclassify stream suit
stream_int = arcpy.Raster(stream_int)
stream_int.readOnly = False

streamsuit = arcpy.sa.Reclassify(stream_int, "Value", "0 1;NODATA 0")

stream_suit = ".\stream_suit.tif"
streamsuit.save(stream_suit)

arcpy.management.Delete(stream_sel)

<Result 'true'>

In [16]:
#Select streams in Lancaster County
munic_sel = arcpy.management.SelectLayerByLocation(munic, "INTERSECT", lancaster)

#Buffer streams by 1 km
municbuff = "munic_buff.shp"
munic_buff = arcpy.analysis.Buffer(munic_sel, municbuff, "5 Kilometers", dissolve_option = 'ALL')

#Erase municbuff from Lancaster
munic_clip = "munic_clip.shp"
arcpy.analysis.Erase(lancaster, munic_buff, munic_clip)

#Convert clip to raster
munic_int = ".\munic_int.tif"
arcpy.conversion.PolygonToRaster(munic_clip, "FID", munic_int)

#Reclassify stream suit
munic_int = arcpy.Raster(munic_int)
munic_int.readOnly = False

municsuit = arcpy.sa.Reclassify(munic_int, "Value", "0 1;NODATA 0")

munic_suit = ".\munic_suit.tif"
municsuit.save(munic_suit)

arcpy.management.Delete(munic_buff, munic_clip)
arcpy.management.Delete(munic_sel)

<Result 'true'>

In [5]:
#Bring in rasters
munic_suit = ".\munic_suit.tif"
dem_suit = ".\\dem_suit.tif"
slope_suit = ".\slope_suit.tif"
stream_suit = ".\stream_suit.tif"
stpark_suit = ".\stpark_suit.tif"

munic_suit = arcpy.Raster(munic_suit)
munic_suit.readOnly = False
dem_suit = arcpy.Raster(dem_suit)
dem_suit.readOnly = False
slope_suit = arcpy.Raster(slope_suit)
slope_suit.readOnly = False
stream_suit = arcpy.Raster(stream_suit)
stream_suit.readOnly = False
stpark_suit = arcpy.Raster(stpark_suit)
stpark_suit.readOnly = False

suitable = munic_suit + dem_suit + slope_suit + stream_suit + stpark_suit

#Reclass suitable
myRemapRange = arcpy.sa.RemapRange([[0, 4, 0], [5, 5, 1]])
suit_rec = arcpy.sa.Reclassify(suitable, 'Value', myRemapRange)

all_suit = ".\suitable.tif"
suit_rec.save(all_suit)